### Import These Libraries

In [ ]:
import matplotlib
%matplotlib inline

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from numpy import expand_dims
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array


#from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import pickle
import cv2
import os

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Lambda
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
import tensorflow as tf

### Initializing Variables 

In [ ]:
EPOCHS = 50
INIT_LR = 1e-3
BS = 32
IMAGE_DIMS = (96, 96, 3)

### Class For Multi Output Multi Label Classification

In [ ]:
class MyModel:
    @staticmethod
    def build_type_branch(inputs, numTypes,
        finalAct="softmax", chanDim=-1):
       
        
      
        x = Conv2D(32, (3, 3), padding="same")(inputs)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(3, 3))(x)
        x = Dropout(0.25)(x)
        x = Conv2D(64, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = Conv2D(64, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        x = Dropout(0.25)(x)
        
        x = Conv2D(32, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(3, 3))(x)
        x = Dropout(0.25)(x)
        x = Conv2D(64, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = Conv2D(64, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        x = Dropout(0.25)(x)
       
        x = Conv2D(128, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = Conv2D(128, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        x = Dropout(0.25)(x)
        
        x = Flatten()(x)
        x = Dense(256)(x)
        x = Activation("relu")(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(numTypes)(x)
        x = Activation(finalAct, name="type_output")(x)
       
        return x
    
    @staticmethod
    def build_color_branch(inputs, numColors, finalAct="softmax",
        chanDim=-1):
        # CONV => RELU => POOL
        x = Conv2D(16, (3, 3), padding="same")(inputs)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(3, 3))(x)
        x = Dropout(0.25)(x)
        # CONV => RELU => POOL
        x = Conv2D(32, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        x = Dropout(0.25)(x)
        # CONV => RELU => POOL
        x = Conv2D(32, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        x = Dropout(0.25)(x)
        
        x = Flatten()(x)
        x = Dense(128)(x)
        x = Activation("relu")(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(numColors)(x)
        x = Activation(finalAct, name="color_output")(x)
       
        return x
    
    @staticmethod
    def build_company_branch(inputs, numCompanies, finalAct="softmax",
        chanDim=-1):
       
        x = Conv2D(16, (3, 3), padding="same")(inputs)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(3, 3))(x)
        x = Dropout(0.5)(x)
       
        x = Conv2D(32, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        x = Dropout(0.5)(x)
        
        x = Conv2D(32, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        x = Dropout(0.5)(x)
        
        x = Conv2D(32, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        x = Dropout(0.5)(x)
        
        
        x = Flatten()(x)
        x = Dense(128)(x)
        x = Activation("relu")(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(numCompanies)(x)
        x = Activation(finalAct, name="company_output")(x)
       
        return x
    
    @staticmethod
    def build_view_branch(inputs, numViews, finalAct="softmax",
        chanDim=-1):
       
        x = Conv2D(16, (3, 3), padding="same")(inputs)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(3, 3))(x)
        x = Dropout(0.25)(x)
       
        x = Conv2D(32, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        x = Dropout(0.25)(x)
        
        x = Conv2D(32, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        x = Dropout(0.25)(x)
        
        x = Flatten()(x)
        x = Dense(128)(x)
        x = Activation("relu")(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(numViews)(x)
        x = Activation(finalAct, name="view_output")(x)
       
        return x
    @staticmethod
    def build_height_branch(inputs, numHeight=1, finalAct="relu",
        chanDim=-1):
        x = Lambda(lambda c: tf.image.rgb_to_grayscale(c))(inputs)
      
        x = Conv2D(32, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(3, 3))(x)
        x = Dropout(0.25)(x)
        x = Conv2D(64, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = Conv2D(64, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        x = Dropout(0.25)(x)
       
        x = Conv2D(128, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = Conv2D(128, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        x = Dropout(0.25)(x)
        
        x = Flatten()(x)
        x = Dense(256)(x)
        x = Activation("relu")(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(numHeight)(x)
        x = Activation(finalAct, name="height_output")(x)
        
        return x
    
    @staticmethod
    def build_scratch_branch(inputs, numScratch=1, finalAct="sigmoid",
        chanDim=-1):
        x = Lambda(lambda c: tf.image.rgb_to_grayscale(c))(inputs)
      
        x = Conv2D(32, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(3, 3))(x)
        x = Dropout(0.25)(x)
        x = Conv2D(64, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = Conv2D(64, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        x = Dropout(0.25)(x)
       
        x = Conv2D(128, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = Conv2D(128, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        x = Dropout(0.25)(x)
        
        x = Flatten()(x)
        x = Dense(256)(x)
        x = Activation("relu")(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(numScratch)(x)
        x = Activation(finalAct, name="scratch_output")(x)
        return x
    
    
    
    @staticmethod
    def build_width_branch(inputs, numWidth=1, finalAct="relu",
        chanDim=-1):
        x = Lambda(lambda c: tf.image.rgb_to_grayscale(c))(inputs)
      
        x = Conv2D(32, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(3, 3))(x)
        x = Dropout(0.25)(x)
        x = Conv2D(64, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = Conv2D(64, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        x = Dropout(0.25)(x)
       
        x = Conv2D(128, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = Conv2D(128, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        x = Dropout(0.25)(x)
        
        x = Flatten()(x)
        x = Dense(256)(x)
        x = Activation("relu")(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(numWidth)(x)
        x = Activation(finalAct, name="width_output")(x)
        
        return x
    @staticmethod
    def build(width, height, numTypes, numColors,numCompanies,numViews =5,finalAct="softmax"):
        
        inputShape = (height, width, 3)
        chanDim = -1
        
        inputs = Input(shape=inputShape)
        typeBranch = MyModel.build_type_branch(inputs,
            numTypes, finalAct=finalAct, chanDim=chanDim
        ) 
        colorBranch = MyModel.build_color_branch(inputs,
            numColors, finalAct=finalAct, chanDim=chanDim)
        companyBranch = MyModel.build_company_branch(
            inputs,
            numCompanies, finalAct=finalAct, chanDim=chanDim
        )
        viewBranch =  MyModel. build_view_branch(inputs,
            numViews, finalAct=finalAct, chanDim=chanDim
        )
        scratchBranch = MyModel.build_scratch_branch(inputs)
        
        model = Model(
            inputs=inputs,
            outputs=[typeBranch, colorBranch
                     #,companyBranch,
                     ##scratchBranch,viewBranch
                    ],
            name="mymodel")
       
        return model
    
       
    
    

### Randomly Shuffle All The Images

In [ ]:
# grab the image paths and randomly shuffle them
print("[INFO] loading images...")
imagePaths = sorted(list(paths.list_images("data")))
random.seed(42)
random.shuffle(imagePaths)
# initialize the data, clothing category labels (i.e., shirts, jeans,
# dresses, etc.) along with the color labels (i.e., red, blue, etc.)
data = []
categoryLabels = []
colorLabels = []

### Image Augmentation

In [ ]:
# path         --> path where all the images are stored
# no_of_IMAGES --> number of augmented images should be created
def augmented_images_generations(path,no_of_images):
    d = os.listdir(path)
     
    for j in range(len(d)):
        (type,color,company,height,width,scratch,view) = d[j].split('_')[:]
        view = view.split('.')[0]
        (type,color,company,height,width,scratch,view) = [x.lower() for x in (type,color,company,height,width,scratch,view)]
        print(type,color,company,height,width,scratch,view)
        img = load_img(os.path.join(path,d[j]))
       
        
        data = img_to_array(img)
        samples = expand_dims(data, 0)
        datagen = ImageDataGenerator(height_shift_range=0.5,zoom_range=0.2,rotation_range=45)
        it = datagen.flow(samples, batch_size=1)
        counter = 1
        os.mkdir(os.path.join("output",type+"_"+color+"_"+company+"_"+scratch+"_"+view))
        for i in range(no_of_images):
            
            batch = it.next()
            image = batch[0].astype('uint8')
            p = os.path.join("output",type+"_"+color+"_"+company+"_"+scratch+"_"+view,str(counter)+".png")
            print(p)
            cv2.imwrite(p,image)
            counter+=1

In [ ]:
augmented_images_generations("data",20)

In [ ]:
os.listdir("output")

### Getting the labels of each Feature (Type , color) from the name of the image where images are stored under the names of their features

In [ ]:
type_labels=[]
color_labels=[]
view_labels=[]
company_labels=[]
scratch_labels=[]
data=[]

for i in os.listdir("output"):
    
    print(i.split('_')[:])
    if(i==".DS_Store"):
        continue
    (type,color,company,scratch,view) = i.split('_')[:]
    view = view.split('.')[0]
    for j in os.listdir("output/"+i):
        image = cv2.imread("output/"+i+"/"+j)
        image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = img_to_array(image)
        data.append(image)
        type_labels.append(type)
        color_labels.append(color)
        view_labels.append(view)
        company_labels.append(company)
        scratch_labels.append(int(scratch))
        
        
        

### Converting them to numpy arrays since CNN's only deal with numpy arrays

In [ ]:
data = np.array(data, dtype="float") / 255.0
print("[INFO] data matrix: {} images ({:.2f}MB)".format(
    len(imagePaths), data.nbytes / (1024 * 1000.0)))

type_labels = np.array(type_labels)
color_labels = np.array(color_labels)
view_labels = np.array(view_labels)
company_labels = np.array(company_labels)
scratch_labels = [int(x) for x in scratch_labels]
scratch_labels = np.array(scratch_labels)

### Encoding the Labels

In [ ]:
print("[INFO] binarizing labels...")
typeLB = LabelBinarizer()
colorLB = LabelBinarizer()
viewLB = LabelBinarizer()
companyLB = LabelBinarizer()

type_labels = typeLB.fit_transform(type_labels)
color_labels = colorLB.fit_transform(color_labels)
view_labels = viewLB.fit_transform(view_labels)
company_labels = companyLB.fit_transform(company_labels)

### Train Test Split

In [ ]:
split = train_test_split(data, type_labels, color_labels,view_labels,company_labels,scratch_labels,
    test_size=0.2, random_state=42)

(trainX, testX, trainTypeY, testTypeY,
    trainColorY, testColorY,trainViewY,testViewY,trainCompanyY,testCompanyY,trainScratchY,testScratchY) = split

### Compiling the Model

In [ ]:
losses = {
    "type_output": "categorical_crossentropy",
    "color_output": "categorical_crossentropy",
    #"company_output":"categorical_crossentropy",
    #"scratch_output":"binary_crossentropy",
    #"view_output":"categorical_crossentropy"
}

lossWeights = {"type_output": 1.0, "color_output": 1.0,
               #"company_output":1.0,
              #"scratch_output":0.1,"view_output":1.0
              }
model = MyModel.build(96,96,len(typeLB.classes_),len(colorLB.classes_),len(companyLB.classes_),len(viewLB.classes_))

print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(optimizer=opt, loss=losses, loss_weights=lossWeights,
    metrics=["accuracy"],run_eagerly=True)

### Training the model over 10 epochs

In [ ]:
EPOCHS = 10

H = model.fit(x=data,
    y={"type_output": type_labels, "color_output": color_labels
       #,"company_output":company_labels,
       #"scratch_output":scratch_labels,
      #"view_output":view_labels
      },
    epochs=EPOCHS,
    verbose=1)

### prediction on new images

In [ ]:
for i in os.listdir("data"):
    
    image = cv2.imread("data/"+i)
    if(i==".DS_Store"):
        continue
    print(i,end="")
    image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    image = img_to_array(image)
    samples = expand_dims(image, 0)
    res = model.predict(samples)

    print(typeLB.classes_[np.argmax(res[0][0])],colorLB.classes_[np.argmax(res[1][0])])

### Saving the model weights

In [ ]:
model.save("model.h5")

In [ ]:
print(typeLB.classes_)

In [ ]:
print(companyLB.classes_)

### Saving the Encoded labels of each feature as pickle file 

In [ ]:
f = open("type_lb.pickle","wb")


f.write(pickle.dumps(typeLB))

f = open("color_lb.pickle","wb")

f.write(pickle.dumps(colorLB))

#f = open("company_lb.pickle","wb")

#f.write(pickle.dumps(companyLB))



In [ ]:
f =  open("type_lb.pickle","rb") 
t=pickle.loads(f.read())

In [ ]:
t.classes_

In [ ]:
def fun():
    return {'path':'123',
    'type':typeLB.classes_[np.argmax(res[0][0])],
    'color':colorLB.classes_[np.argmax(res[1][0])],
    'company':companyLB.classes_[np.argmax(res[2][0])]
    }

In [ ]:

import tensorflow as tf

from keras.models import model_from_json
model.save('model2.model',save_format="h5")

from keras.models import load_model

x=load_model('model2.model',custom_objects={"": tf})

In [ ]:
for i in os.listdir("data"):
    
    image = cv2.imread("data/"+i)
    if(i==".DS_Store"):
        continue
    print(i,end="")
    image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    image = img_to_array(image)
    samples = expand_dims(image, 0)
    res = x.predict(samples)

    print(typeLB.classes_[np.argmax(res[0][0])],colorLB.classes_[np.argmax(res[1][0])])

In [ ]:
from flask import request
from flask import jsonify
from flask_cors import CORS, cross_origin
from flask import render_template
from flask import Flask
import base64

In [ ]:
app = Flask(__name__)

In [ ]:
tem = [{'path': 'SIH_Video/7/i5_1.png', 'type': 'trolley', 'color': 'violet'}, {'path': 'SIH_Video/7/i6_1.png', 'type': 'trolley', 'color': 'violet'}, {'path': 'SIH_Video/7/i6_2.png', 'type': 'trolley', 'color': 'violet'}, {'path': 'SIH_Video/7/i6_3.png', 'type': 'trolley', 'color': 'violet'}, {'path': 'SIH_Video/7/i8_1.png', 'type': 'trolley', 'color': 'violet'}, {'path': 'SIH_Video/7/i8_2.png', 'type': 'trolley', 'color': 'violet'}, {'path': 'SIH_Video/7/i9_1.png', 'type': 'trolley', 'color': 'grey'}, {'path': 'SIH_Video/7/i10_1.png', 'type': 'trolley', 'color': 'violet'}, {'path': 'SIH_Video/7/i11_1.png', 'type': 'trolley', 'color': 'violet'}, {'path': 'SIH_Video/7/i12_1.png', 'type': 'trolley', 'color': 'violet'}, {'path': 'SIH_Video/7/i12_2.png', 'type': 'trolley', 'color': 'violet'}, {'path': 'SIH_Video/7/i13_1.png', 'type': 'trolley', 'color': 'grey'}, {'path': 'SIH_Video/7/i15_1.png', 'type': 'trolley', 'color': 'grey'}]

In [ ]:
for i in tem:
    print(i['color'])